### This notebook contains a simple implementation to visualize `MTR` weighted fibers interactively
* For the transformations, the same convention in the Octave/MATLAB scripts were followed 

**Note:** Implementations in this repository are NOT intended for research, for demonstration ONLY.

In [ ]:
import scipy.io as sio
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy import ndimage

In [ ]:
# Do not forget to change file directory if running locally
fname = '/home/jovyan/work/Eda_Data/S2/visdata/inputVolumes.mat'
mri = sio.loadmat(fname, struct_as_record=True)

In [ ]:
fname2 = '/home/jovyan/work/Eda_Data/S2/visdata/interpolated_500.mat'
tfibers = sio.loadmat(fname2, struct_as_record=True)
fibers = tfibers['interpolatedFibers'] # fibers[:,0] fibers[:,1] fibers[:,2]

In [ ]:
mtOn = mri['data'][0,0]['mtOn']
volume = mtOn.reshape(256,256,50)

In [ ]:
header = mri['header'][0,0]['mtOn']
spacing = np.diag(header['spacing'][0][0][0])
origin = header['origin'][0][0][0]
direction = np.diag([1,1,1])

In [ ]:
def getXyz(direction,origin,spacing,ijk):
    x = np.matmul(direction,spacing)
    v = np.matmul(x,ijk) + origin
    return v
    
def getDomain(xSize,ySize, zPoint):
    lwr = getXyz(direction,origin,spacing,[1,1,zPoint])
    upr = getXyz(direction,origin,spacing,[xSize,ySize,zPoint])
    x = np.linspace(lwr[0], upr[0], xSize)
    y = np.linspace(lwr[1], upr[1], ySize)
    z = lwr[2]
    return x,y,z

def getTrace(fib,scalar,cmn,cmx):
    trace=go.Scatter3d(
     x=fib[:,0], y=fib[:,1], z=fib[:,2],
     mode ='lines', 
     projection = dict(x = dict(show = False),
                  y = dict(show = False),
                  z = dict(show = False)),   
     opacity =0.7,
     text = fib[:,scalar+2] ,    
     line=dict(
        colorscale='Viridis',
        width=4,
        color=fib[:,scalar+2],
        cmin=cmn,
         cmax=cmx  
        )
    )

    return trace

In [ ]:
x,y,z = getDomain(256,256,25)

def getPlot(sclr,cmn,cmx):
    fig = go.Figure()
    r, c = cc[0].shape
    fig.add_trace(go.Surface(
        z=z* np.ones((256, 256)),
        x=x,y=y,
        opacity = 0.85,
        showscale=False,
        surfacecolor= ndimage.rotate(np.fliplr(cc[:,:,30]),90,reshape=False),
        colorscale='Gray',
        cmin=0, cmax=3000,
        colorbar=dict(thickness=20, ticklen=4)
        ))

    for i in range(500):
        fig.add_trace(getTrace(np.array(fibers[0,i]['matrix']),sclr,cmn,cmx))
    
    return fig    

<center> <h1> ⚠️ </h1> <center>
<center> <h3 style="color:red;">I did not calculate the exact transformation from <code>dwi</code> to <code>MTR</code> space! </h3></center>

<center> <h4 style="color:coral;"> I tweaked the (inverse of the) affine transformation created by DSI studio after importing anatomical images and used it for transforming fiber node coordinates to <u style="color:red;">roughly align</u> with a single slice.  </h4></center><br>


<center> Therefore alignment of these fibers with the <code>MTR</code> domain is far from convenient. 

#### Color range of the nodal  `MTR` values are adjusted on purpose to highlight the consequences of imperfect alignment

You can tell by the appearence of the ventricles pattern on the <code>MTR weighted</code> tracts that are actually crossing corpus callosum in `dwi` space.

In [ ]:
fig = getPlot(1,-50,18) # Nodal MTR
fig.layout.showlegend = False
fig.show()

In [ ]:
fig2 = getPlot(2,-50,-20) # Avg MTR
fig2.layout.showlegend = False
fig2.show()